In [1]:
# 导入相关库
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import datetime
import random
import os
import rasterio
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle


In [2]:

class AFF(nn.Module):
    '''
    多特征融合 AFF
    '''

    def __init__(self, channels, r=4):
        super(AFF, self).__init__()
        inter_channels = int(channels // r)

        # 局部注意力模块
        self.local_att = nn.Sequential(
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )

        # 全局注意力模块
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )

        # Sigmoid函数用于生成注意力权重
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, residual):
        # 计算融合的输入
        xa = x + residual

        # 计算局部和全局注意力
        xl = self.local_att(xa)
        xg = self.global_att(xa)

        # 将局部和全局注意力相加并应用sigmoid函数得到最终的注意力权重
        xlg = xl + xg
        wei = self.sigmoid(xlg)

        # 使用注意力权重融合原始输入和残差
        xo = 2 * x * wei + 2 * residual * (1 - wei)
        return xo


In [3]:
class ResNet50Economic(nn.Module):
    def __init__(self, econ_features_dim=6, num_classes=1):
        super(ResNet50Economic, self).__init__()
        
        # 使用ResNet50作为图像特征提取器
        weights = ResNet50_Weights.DEFAULT
        self.resnet = resnet50(weights=weights)

        # 初始化新的7通道卷积层
        new_conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # 方法1: 随机初始化所有通道
        #torch.nn.init.kaiming_normal_(new_conv1.weight, mode='fan_out', nonlinearity='relu')

        # 或者方法2: 复制原始权重到前3个通道，其余通道随机初始化
        original_weights = self.resnet.conv1.weight.data
        new_conv1.weight.data[:, :3, :, :] = original_weights
        torch.nn.init.kaiming_normal_(new_conv1.weight[:, 3:, :, :], mode='fan_out', nonlinearity='relu')

        # 将新的卷积层赋值给模型
        self.resnet.conv1 = new_conv1

        # 将ResNet-50的全连接层替换为恒等映射
        self.resnet.fc = nn.Identity()
        
        # 定义用于处理经济特征的线性层
        self.econ_fc = nn.Linear(econ_features_dim, 2048)

        # 定义AFF模块
        self.aff = AFF(channels=2048)  # 假设ResNet-50的输出通道数为2048

        # 定义最终的全连接层，用于输出预测结果
        self.final_fc = nn.Linear(4096, num_classes)

    def forward(self, images, econ_features):
        # 通过ResNet-50提取图像特征
        img_features = self.resnet(images)

        # 通过线性层处理经济特征
        econ_features = self.econ_fc(econ_features)
        econ_features = econ_features.unsqueeze(-1).unsqueeze(-1)
        img_features = img_features.unsqueeze(-1).unsqueeze(-1)
        #print("输入图像尺寸:", img_features.shape)
        #print("输入经济特征尺寸:", econ_features.shape)
        # 使用AFF模块融合图像和经济特征
        #combined_features = self.aff(img_features, econ_features)
        # 使用直接拼接的方式融合图像和经济特征
        combined_features = torch.cat((img_features, econ_features), dim=1)
        #combined_features = img_features
        #print("融合后尺寸:", combined_features.shape)
        # 通过自适应平均池化层将特征映射到一维
        combined_features = F.adaptive_avg_pool2d(combined_features, (1, 1))
        combined_features = torch.flatten(combined_features, 1)
        # 通过最终的全连接层获取预测结果
        out = self.final_fc(combined_features)
        return out

In [4]:
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss, total_ssr, total_tss, total_samples = 0, 0, 0, 0
    actuals, predictions, years = [], [], []
    geo_fips = []
    with torch.no_grad():
        for images, econ_features,targets,year, fips in loader:
            images = images.to(device)
            econ_features = econ_features.to(device)
            targets = targets.to(device)

            targets = targets.float().unsqueeze(1)

            outputs = model(images, econ_features)
            loss = criterion(outputs, targets)
            # 收集实际值、预测值和 GeoFIPS
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())
            years.extend(year.cpu().numpy())  # 添加年份信息 
            geo_fips.extend(fips.cpu().numpy())
            # 计算准确度，仅包含非零目标
            non_zero_targets = targets != 0
            if non_zero_targets.any():
                total_loss += loss.item() * targets.size(0)
                residuals = outputs - targets
                total_ssr += torch.sum(residuals ** 2).item()
                total_tss += torch.sum((targets - torch.mean(targets)) ** 2).item()
                total_samples += targets.size(0)

    test_loss = total_loss / total_samples
    test_r2_score = 1 - (total_ssr / total_tss)
    print(f"Test Loss: {test_loss:.4f}, Test R2 Score: {test_r2_score:.4f}")
    return test_loss, test_r2_score, np.array(actuals), np.array(predictions), np.array(years), np.array(geo_fips)

In [5]:
def train_model(model, train_loader, val_loader, optimizer,scheduler, criterion, device, num_epochs):
    model.to(device)
    train_losses, val_losses = [], []
    train_r2_scores, val_r2_scores = [], []
    min_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_loss, total_ssr, total_tss, total_samples = 0, 0, 0, 0

        tqdm_train_loader = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)
        torch.cuda.empty_cache()
        for images, econ_features, targets,year, fips in tqdm_train_loader:
            images = images.to(device)
            econ_features = econ_features.to(device)
            targets = targets.to(device)

            targets = targets.float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images, econ_features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * targets.size(0)

            # Calculate SSR and update TSS
            residuals = outputs - targets
            total_ssr += torch.sum(residuals ** 2).item()
            total_tss += torch.sum((targets - torch.mean(targets)) ** 2).item()
            total_samples += targets.size(0)

            # 更新 tqdm 进度条
            tqdm_train_loader.set_postfix(train_loss=loss.item(), refresh=True)
        torch.cuda.empty_cache()
        train_loss = total_loss / total_samples
        train_r2_score = 1 - (total_ssr / total_tss)
        train_losses.append(train_loss)
        train_r2_scores.append(train_r2_score)

        val_loss, val_r2_score, actuals, predictions, years, geo_fips = evaluate_model(model, val_loader, criterion, device)


        val_losses.append(val_loss)
        val_r2_scores.append(val_r2_score)
        
        # 使用 ReduceLROnPlateau 调整器
        scheduler.step(val_loss)
        
        # 检查并更新最佳验证损失
        if val_loss < min_val_loss:
            print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train R2: {train_r2_score:.4f}, Val Loss: {val_loss:.4f}, Val R2: {val_r2_score:.4f}')
            print(f'val loss decreased ({min_val_loss:.6f} -> {val_loss:.6f}). Saving model.')
            min_val_loss = val_loss
            # 保存模型
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, 'naffbest_level_model_nof.pth')

    return train_losses, val_losses, train_r2_scores, val_r2_scores

In [6]:

# 加载处理后的数据集
def load_processed_dataset(file_path):
    with open(file_path, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

processed_train_dataset = load_processed_dataset("comts_train_dataset.pkl")
processed_val_dataset = load_processed_dataset("comts_val_dataset.pkl")
processed_test_dataset = load_processed_dataset("comts_test_dataset.pkl")

In [7]:
# 创建数据加载器
batch_size = 8  # 或根据您的需求调整批处理大小
train_loader = DataLoader(processed_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(processed_val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(processed_test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# 确保选择了正确的设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型权重
model = ResNet50Economic(econ_features_dim=11, num_classes=1)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
checkpoint = torch.load(('E:\\FYP\\fypcode\\Code\\Level_Final\\naffbest_level_model_nof.pth'))
model_state_dict = checkpoint['model_state_dict']
optimizer_state_dict = checkpoint['optimizer_state_dict']
optimizer.load_state_dict(optimizer_state_dict)
model.load_state_dict(model_state_dict)


<All keys matched successfully>

In [9]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")

Layer: resnet.conv1.weight | Size: torch.Size([64, 7, 7, 7]) | Values : tensor([[[[-7.4457e-03, -3.1783e-03,  3.7353e-02, -6.3583e-02,  4.7936e-02,
           -2.0325e-02,  8.8140e-03],
          [-5.7435e-02,  4.4709e-02,  7.7509e-02, -1.7870e-01,  8.8442e-02,
            2.9346e-02, -5.8331e-02],
          [ 6.8356e-02, -2.7044e-01,  4.0348e-01, -1.3433e-01, -1.6491e-01,
            2.1868e-01, -7.2909e-02],
          [ 2.5066e-02,  1.2906e-01, -6.6479e-01,  1.1661e+00, -9.2312e-01,
            2.8766e-01, -3.7066e-02],
          [-1.0874e-01,  3.8148e-01, -4.5487e-01, -1.0326e-01,  6.8366e-01,
           -5.7855e-01,  2.2461e-01],
          [ 2.5698e-02, -1.7703e-01,  6.4375e-01, -9.1184e-01,  5.2644e-01,
           -4.9317e-02, -6.8082e-02],
          [ 4.5281e-02, -1.3072e-01,  1.7864e-02,  2.5106e-01, -3.5753e-01,
            1.8976e-01, -2.2302e-02]],

         [[ 8.9197e-03,  4.8768e-03, -1.5356e-02, -2.8934e-02,  8.6949e-02,
           -6.5541e-02,  1.6895e-02],
          [-3.

In [10]:
# 获取第一个样本
image, econ_features, label,years, geo_fips = processed_train_dataset[0]
#输出dataset的years
print(years)

2004


In [11]:



print(image.shape)  # 检查图像的形状
# 显示经济特征
print("Economic Features:", econ_features)


torch.Size([7, 512, 512])
Economic Features: tensor([0.9068, 0.9009, 0.8279, 0.3349, 0.5119, 0.6254, 0.5122, 0.5047, 1.0000,
        0.5289, 0.2398])


In [ ]:


# 训练模型，并接收四个返回值
train_losses, val_losses, train_r2_scores, val_r2_scores = train_model(model, train_loader, test_loader, optimizer,scheduler, criterion, device, num_epochs=50)


In [14]:
# 使用evaluate_model函数获得实际值和预测值
test_loss, test_r2_score, actuals, predictions, years, geo_fips = evaluate_model(model, train_loader, criterion, device)

Test Loss: 0.0019, Test R2 Score: 0.9994


In [ ]:
geo_fips = geo_fips.flatten()
years = years.flatten()
actuals = actuals.flatten()
predictions = predictions.flatten()


# 创建DataFrame
results_df = pd.DataFrame({
    'GeoFIPS': geo_fips,
    'Year': years,
    'Actuals': actuals,
    'Predictions': predictions
})

# 保存为CSV
results_df.to_csv('0nftrain_model_results.csv', index=False)